In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la

# Actividad 08: Algebra Lineal y Matrices

---
### Profesor: Juan Marcos Marín
### Nombre: Jose Nicolas Rodriguez Montoya
*Métodos computacionales*

---

#1
Escriba tres matrices aleatorias $A$, $B$ y $C$ de $3\times 3$, y demuestre las siguientes relaciones

- $ \mathbf{A}\mathbf{B} \neq \mathbf{B}\mathbf{A} $, en general.
- $ (\mathbf{A}\mathbf{B})\mathbf{C} = \mathbf{A}(\mathbf{B}\mathbf{C}) $.
- $ \mathbf{A}(\mathbf{B} + \mathbf{C}) = \mathbf{A}\mathbf{B} + \mathbf{A}\mathbf{C} $.
- $ (\mathbf{A} + \mathbf{B})\mathbf{C} = \mathbf{A}\mathbf{C} + \mathbf{B}\mathbf{C} $.
- $ (\mathbf{A}\mathbf{B})^\top = \mathbf{B}^\top \mathbf{A}^\top $.
- $ \det(\mathbf{A}\mathbf{B}) = \det(\mathbf{A}) \det(\mathbf{B}) $.
- $ (\mathbf{A}^\top)^\top = \mathbf{A} $.
- $ (c\mathbf{A})^\top = c\mathbf{A}^\top $.
- $ (\mathbf{A} + \mathbf{B})^\top = \mathbf{A}^\top + \mathbf{B}^\top $.



In [ ]:
from numpy import linalg as la

A = np.random.rand(3, 3)  # Definimos 3 matrices con entradas flotantes aleatorias
B = np.random.rand(3, 3)
C = np.random.rand(3, 3)

conmutatividad = np.allclose(A@B, B@A) # Para cada caso definimos una variable Booleana que nos idra si la condicion se ccumple o no
print(f'la conmutatividad es {conmutatividad}')

asociatividad = np.allclose((A@B)@C, A@(B@C))
print(f'la asociatividad es {asociatividad}')

distributividad = np.allclose(A@(B+C), (A@B) + (A@C))
print(f'la distributividad es {distributividad}')

distributividad_2 = np.allclose((A+B)@C, A@C + B@C)
print(f'la distributividad es {distributividad}')

transpose_distribute = np.allclose((A@B).T, (B.T) @ (A.T))
print(f'la distribucion de transpuesta es {transpose_distribute}')

distribute_det = np.allclose(la.det(A@B), la.det(A)*la.det(B))
print(f'la distribucion de determinante es {distribute_det}')

inversibilidad_transpose = np.allclose((A.T).T, A)
print(f'la inversibilidad de transpuesta es {inversibilidad_transpose}')

escalar_transpose = np.allclose((3*A).T, 3*(A.T))
print(f'la escalar de transpuesta es {escalar_transpose}')

distribute_transpose = np.allclose((A+B).T, (A.T) + (B.T))
print(f'la distribucion de transpuesta es {distribute_transpose}')

la conmutatividad es False
la asociatividad es True
la distributividad es True
la distributividad es True
la distribucion de transpuesta es True
la distribucion de determinante es True
la inversibilidad de transpuesta es True
la escalar de transpuesta es True
la distribucion de transpuesta es True


#2

El **Teorema de Laplace** es un método para calcular el determinante de una matriz cuadrada, particularmente útil para matrices de orden mayor a 2. Este teorema se basa en la expansión del determinante por los elementos de una fila o una columna cualquiera.



$$
\det(A) = \sum_{j=1}^n (-1)^{1+j} a_{1j} M_{1j}
$$

donde:
- $a_{1j}$ es el elemento de la primera fila y columna $j$.
- $M_{1j}$ es el menor asociado al elemento $a_{1j}$, es decir, el determinante de la submatriz de $3 \times 3$ que se obtiene al eliminar la fila 1 y la columna $j$.
- $(-1)^{1+j}$ es el signo correspondiente al cofactor del elemento $a_{1j}$.

Podemos realizar una función recursiva para el cálculo del determinante, sabiendo que el valor del determinante de una matriz de orden uno es el único elemento de esa matriz, y el de una matriz de orden superior a uno es la suma de cada uno de los elementos de una fila o columna por los Adjuntos a ese elemento, como en la función recursiva se emplea la misma función definida el cálculo lo haremos por Menor complementario, un ejemplo desarrollado por la primera fila sería:

$$
   \det (A_{j,j}) =
   \left \{
   \begin{array}{llcl}
      si & j = 1 & \to & a_{1,1} \\
                                 \\
      si & j > 1 & \to & \displaystyle \sum_{k=1}^j \; (-1)^{(1+k)} \cdot a_{1,k} \cdot \det( \alpha_{1,k})
   \end{array}
   \right .
$$

Realice una función que encuentre el determinante de una matriz usando la recursividad aqui planteada, explique explicitamente su código

In [ ]:
def laplace_theorem(M):
  '''
  Regresa el determinante de una Matriz M usando el teorema de Laplace

  Entradas:
    M (matriz): matriz cuadrada ndarray

  Salida:
    Coeff: determinante de la función -> float

  '''
  n = np.shape(M)[0]  #le decimos a la maquina cuantas filas tiene nuestra matriz
  if np.shape(M)[0] != np.shape(M)[1]:
    raise ValueError('La matriz no es cuadrada')

  #Caso base
  if n == 1:
    return M.item() #cuando es de un solo elemento  y habiando comprobado que es cuadrada el determinante sera el unico elemento de la matriz

  det = 0 #Inicializamos una suma en 0

  for j in range(n):
        # Crear la submatriz eliminando fila 0 y columna j
        menor = np.delete(np.delete(M, 0, axis=0), j, axis=1)

        # Calcular el cofactor: (-1)^(i+j) donde i=0 (primera fila)
        cofactor = (-1) ** (0 + j) * M[0, j]

        # Llamada recursiva
        det += cofactor * laplace_theorem(menor)

  return det



In [ ]:
A = np.array([[1,2,5],
              [4,6,3],
              [7,8,0]], dtype= float)


laplace_theorem(A)

np.float64(-32.0)

#3 Método de Gauss - Seidel

Sea \$A\in\mathbb{R}^{n\times n}\$ no singular y sea \$b\in\mathbb{R}^n\$.
Descomponga \$A\$ como

$$
A \;=\; D \;+\; L \;+\; U,
$$

donde

* \$D\$ es la matriz diagonal de \$A\$,
* \$L\$ es la parte estrictamente triangular inferior,
* \$U\$ es la parte estrictamente triangular superior.

El algoritmo de Gauss - Seidel reorganiza el sistema \$Ax=b\$ como

$$
x \;=\; (D+L)^{-1}\bigl(b \;-\; Ux\bigr),
$$

y genera la sucesión

$$
x_i^{(k+1)}
= \frac{1}{a_{ii}}
\Bigl(b_i - \sum_{j<i} a_{ij}\,x_j^{(k+1)} - \sum_{j>i} a_{ij}\,x_j^{(k)}\Bigr),
\qquad i=1,\dots,n.
$$

Implemente una función `gauss_seidel(A, b, tol=1e-7, max_iter=100)` que:
   * Realice las iteraciones hasta que
     $\lVert x^{(k+1)}-x^{(k)}\rVert_\infty<\text{tol}$
     o se alcance `max_iter`;
   * devuelva el vector solución aproximado \$x\$, el número de iteraciones realizadas y la norma del último residuo.

Incluya una documentación clara.

Luego,

   * Genere una matriz aleatoria \$5\times5\$ (por ejemplo, con `np.random.rand`) y un vector \$b\$ aleatorio.
   * Resuelva \$Ax=b\$ con su función; calcule el error relativo frente a `numpy.linalg.solve`.
   * Estime igualmente el error respecto a la solución obtenida mediante \$x=A^{-1}b\$ (usando `numpy.linalg.inv`).
   * Presente las normas de los residuos y los errores relativos.

In [ ]:
def gauss_seidel(A, b, tol=1e-7, max_iter=100):
    # Verificar que A es cuadrada
    n, m = A.shape
    if n != m:
        raise ValueError("La matriz A debe ser cuadrada.")

    # Verificar que b tiene la dimensión correcta
    if len(b) != n:
        raise ValueError("Las dimensiones de A y b no son compatibles.")

    # Inicializar el vector solución
    x = np.zeros_like(b, dtype=np.float64)
    x_prev = np.copy(x)

    # Iteraciones de Gauss-Seidel
    for k in range(max_iter):
        for i in range(n):
            # Calcular las sumas separando los términos ya actualizados y no actualizados
            sum1 = np.dot(A[i, :i], x[:i])  # Suma de términos ya actualizados (j < i)
            sum2 = np.dot(A[i, i+1:], x_prev[i+1:])  # Suma de términos no actualizados (j > i)

            # Actualizar la componente i-ésima del vector solución
            x[i] = (b[i] - sum1 - sum2) / A[i, i]

        # Verificar convergencia
        diff_norm = np.linalg.norm(x - x_prev, np.inf)
        if diff_norm < tol:
            break

        # Actualizar x_prev para la siguiente iteración
        x_prev = np.copy(x)

    # Calcular la norma del residuo final
    residual = np.dot(A, x) - b
    residual_norm = np.linalg.norm(residual, np.inf)

    return x, k+1, residual_norm

In [ ]:
# Crear una matriz diagonalmente dominante
M = np.random.rand(5,5)
for i in range(5):
    M[i,i] = np.sum(np.abs(M[i,:])) + 1  # Hacemos la diagonal dominante

b = np.random.rand(5)


print("Solución numpy:", np.linalg.solve(M,b))
print("Solución Gauss-Seidel:", gauss_seidel(M,b)[0])
print("Solucion con la inversa de M:", np.linalg.inv(M) @ b)
print("Error relativo:", np.linalg.norm(np.linalg.solve(M,b) - gauss_seidel(M,b)[0], np.inf) / np.linalg.norm(np.linalg.solve(M,b), np.inf))

Solución numpy: [ 0.24522102 -0.05364504  0.19720506  0.02766589  0.26995875]
Solución Gauss-Seidel: [ 0.24522102 -0.05364504  0.19720506  0.02766588  0.26995875]
Solucion con la inversa de M: [ 0.24522102 -0.05364504  0.19720506  0.02766589  0.26995875]
Error relativo: 4.78489772545916e-09


In [ ]:
def gauss_seidel(A, b, tol=1e-7, max_iter=100):
    """
    Implementa el método de Gauss-Seidel para resolver sistemas lineales Ax = b

    Entradas:
        A (matriz): matriz cuadrada
        b (array): vector de dimensión n
        tol (float): tolerancia para criterio de convergencia
        max_iter (int): número máximo de iteraciones

    Salidas:
        x: vector solución
        k: número de iteraciones realizadas
        residual: norma del residuo
    """
    # Verificar dimensiones
    if A.shape[0] != A.shape[1]:
        raise ValueError("La matriz A debe ser cuadrada")
    if A.shape[0] != b.shape[0]:
        raise ValueError("Dimensiones incompatibles entre A y b")

    n = A.shape[0]
    x = np.zeros_like(b)  # Solución inicial

    for k in range(max_iter):
        x_old = x.copy()  # Guardar la iteración anterior

        # Actualizar cada componente de x
        for i in range(n):
            sum1 = np.dot(A[i, :i], x[:i])  # Suma con los x_i ya actualizados
            sum2 = np.dot(A[i, i+1:], x_old[i+1:])  # Suma con los x_i no actualizados
            x[i] = (b[i] - sum1 - sum2) / A[i, i]

        # Calcular diferencia entre iteraciones y residuo
        diff = np.linalg.norm(x - x_old, np.inf)
        residual = np.linalg.norm(b - np.dot(A, x), np.inf)

        # Verificar criterio de convergencia
        if diff < tol:
            break

    return x, k, residual

In [ ]:
#generamos la matriz aleatoria
M = np.random.rand(5, 5)
#generamos el vector aleatorio
b = np.random.rand(5)

sol_gs = gauss_seidel(M, b)
sol_sol = np.linalg.solve(M, b)
sol_inv = np.linalg.inv(M) @ b

Relative_error_sol = abs(sol_sol - sol_gs[0]) / abs(sol_sol)
Relative_error_inv = abs(sol_inv - sol_gs[0]) / abs(sol_inv)

Relative_error_inv


array([1.05052424e-04, 9.98149887e-05, 1.42314245e-04, 1.12702051e-04,
       8.02648659e-05])

#4 Método de potencias para el valor propio dominante

Sea \$A\in\mathbb{R}^{n\times n}\$ diagonalizable con valor propio dominante \$\lambda\_{\max}\$ (en magnitud) y vector propio asociado \$v\_{\max}\$.

El método de potencias genera, a partir de un vector inicial \$q^{(0)}\neq 0\$, la sucesión

$$
q^{(k+1)} \;=\; \frac{A\,q^{(k)}}{\lVert A\,q^{(k)}\rVert_2},
\qquad
\lambda^{(k+1)} \;=\; (q^{(k+1)})^{\!\top} A\, q^{(k+1)},
$$

que converge a \$v\_{\max}/\lVert v\_{\max}\rVert\_2\$ y a \$\lambda\_{\max}\$ respectivamente, bajo hipótesis estándar.

Implemente `power_method(A, tol=1e-7, max_iter=1000)` que:

   * Acepte matrices reales cuadradas,
   * Devuelva \$\lambda\_{\max}\$, el vector propio normalizado \$v\_{\max}\$, el número de iteraciones y la última variación relativa de \$\lambda\$,
   * detenga la iteración cuando
     $\bigl|\lambda^{(k+1)}-\lambda^{(k)}\bigr|<\text{tol}\,|\lambda^{(k+1)}|$
     o se alcance `max_iter`.

Luego,
   * Genere una matriz simétrica aleatoria \$6\times6\$ (por ejemplo, \$A = (M+M^\top)/2\$ con \$M\$ aleatoria).
   * Aplique su `power_method` y compare \$\lambda\_{\max}\$ y \$v\_{\max}\$ con los resultados de `numpy.linalg.eig`.

In [ ]:
def power_method(A, tol=1e-7, max_iter=1000):
    """
    Calcula el valor propio dominante y el vector propio asociado de una matriz.

    Args:
        A (np.ndarray): Matriz cuadrada.
        tol (float): Tolerancia para la convergencia.
        max_iter (int): Número máximo de iteraciones.

    Returns:
        tuple: (valor propio dominante, vector propio, número de iteraciones, variación relativa)
    """
    if A.shape[0] != A.shape[1]:
        raise ValueError("La matriz A debe ser cuadrada")

    q = np.random.rand(A.shape[0])
    q = q / np.linalg.norm(q)
    lambda_ = 0

    for i in range(max_iter):
        q_new = A @ q
        q_new = q_new / np.linalg.norm(q_new)

        lambda_new = q_new.T @ A @ q_new

        if lambda_ != 0 and abs(lambda_new - lambda_) / abs(lambda_new) < tol:
            return lambda_new, q_new, i + 1, abs(lambda_new - lambda_) / abs(lambda_new)

        q = q_new
        lambda_ = lambda_new

    raise RuntimeError('Máximas iteraciones alcanzadas sin convergencia')

In [ ]:
M = np.random.rand(6,6)
A = (M+(M.T))/2
A

print(f'\nEl autovalor más grande encontrado con el método de potencias fue {power_method(A)[0]}')

print(f'\nel autovalor más grande encontrado: {np.linalg.eig(A)[0][0]}')

print(f'\nlos autovectores encontrados con el metodo de potencias fueron {power_method(A)[1]}')

print(f'\nlos autovectores encontrados con numpy fueron {np.linalg.eig(A)[1][:,0]}')

print(f'\nla variación relativa fue {power_method(A)[3]}')

print(f'\ntardo {power_method(A)[2]} iteraciones')


El autovalor más grande encontrado con el método de potencias fue 3.0603712949305857

el autovalor más grande encontrado: 3.0603712973584445

los autovectores encontrados con el metodo de potencias fueron [0.37852348 0.37416574 0.41942932 0.35166856 0.44737405 0.46581617]

los autovectores encontrados con numpy fueron [-0.37850298 -0.37420448 -0.41940565 -0.35168625 -0.44740292 -0.46578192]

la variación relativa fue 1.1085832518295781e-08

tardo 7 iteraciones


#5

Verifique que cualquier matriz hermitiana de 2 × 2 $ L $ puede escribirse como una suma de cuatro términos:

$$ L = a\sigma_x + b\sigma_y + c\sigma_z + dI $$

donde $ a $, $ b $, $ c $ y $ d $ son números reales.

Las cuatro matrices de Pauli son:

$$ \sigma_x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad \sigma_y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad \sigma_z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}, \quad I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} $$




In [ ]:
# Definir las matrices de Pauli y la identidad
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
I = np.array([[1, 0], [0, 1]], dtype=complex)

# Crear una matriz hermitiana general de 2x2 (ejemplo)
def random_hermitian_matrix():
    # Elementos reales y complejos aleatorios
    a = np.random.rand()  # a real (diagonal)
    d = np.random.rand()  # d real (diagonal)
    b_real = np.random.rand()
    b_imag = np.random.rand()
    b = b_real + 1j * b_imag  # b complejo (fuera de diagonal)
    # Construir la matriz hermitiana: L = [[a, b], [conj(b), d]]
    L = np.array([[a, b], [np.conj(b), d]], dtype=complex)
    return L

L = random_hermitian_matrix()
print("Matriz hermitiana L generada:\n", L)

# Resolver para los coeficientes a, b, c, d (reales)
# Usamos las propiedades de traza de las matrices de Pauli:
# tr(sigm_i sigm_j) = 2 d_ij, tr(sigm_i) = 0, tr(I) = 2
# Por lo tanto:


a = np.trace(L @ sigma_x) / 2
b = np.trace(L @ sigma_y) / 2
c = np.trace(L @ sigma_z) / 2
d = np.trace(L @ I) / 2

# Los coeficientes deben ser reales porque L es hermitica
a, b, c, d = np.real(a), np.real(b), np.real(c), np.real(d)
print("\nCoeficientes encontrados:")
print(f"a = {a:.4f}, b = {b:.4f}, c = {c:.4f}, d = {d:.4f}")

Matriz hermitiana L generada:
 [[0.04443202+0.j         0.40941173+0.11292281j]
 [0.40941173-0.11292281j 0.95719871+0.j        ]]

Coeficientes encontrados:
a = 0.4094, b = -0.1129, c = -0.4564, d = 0.5008


# 6

Haga un breve resumen en Markdown de las funciones y métodos más relevantes para algebra lineal usando Python. Emplee ejemplos.

## Librerías principales
- **NumPy**: Operaciones básicas con arrays y matrices.
- **SciPy**: Funciones avanzadas de álgebra lineal.

## Funciones y métodos clave

### Creación de matrices
```python
A = np.array([[1, 2], [3, 4]])  # Matriz 2x2
B = np.eye(3)                   # Matriz identidad
Operaciones básicas
python
A @ B               # Multiplicación matricial
np.linalg.inv(A)    # Inversa de matriz
Resolución de sistemas
python
x = np.linalg.solve(A, b)  # Solución de Ax = b
Descomposiciones
python
np.linalg.eig(A)    # Autovalores/autovectores
np.linalg.svd(A)    # Descomposición SVD
Normas y determinante
python
np.linalg.det(A)    # Determinante
np.linalg.norm(A)   # Norma de matriz
